# Benchmarks

## Prep 

In [54]:
from cnn.utils.graph_manipulation import load_frozen_graph, predict_from_frozen
from cnn.utils.dataset import load_cifar10

In [ ]:
pyperf = !find ~ -iname pyperf 2>/dev/null
pyperf = pyperf[0]

In [35]:
nets = ["dense", "conv", "medium"]
names = [[n + "_opt", n + "_quant"] for n in nets]
frozen_nets = [["models/" + n + ".bp"  for n in both] for both in names]
results_files = [["results/" + n + ".json"  for n in both] for both in names]

## Check accuracy 

In [38]:
_, _, x_test, t_test = load_cifar10()

In [ ]:
for pb_files in frozen_nets:
    graph = load_frozen_graph(pb_file)
    out = predict_from_frozen()

## Time comparisons 

In [ ]:
for r in results_files:
    !{pyperf} show {r[0]} --quiet

In [ ]:
for r in results_files:
    !{pyperf} compare_to {r[0]} {r[1]} --table